#Function and Class Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import fnmatch

import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)   # Mount google drive in order to access files

Mounted at /content/drive


If you get the following output, you're all set!

```
Mounted at /content/drive
```



#Separate Tepal Function

In [ ]:
from skimage.util import img_as_bool, img_as_ubyte
from skimage.morphology import erosion as erode
from skimage.morphology import closing
from skimage.morphology import disk

from scipy.ndimage import label as scipylabel

from skimage.measure import regionprops

def separate_tepals(im):
  
  # Remove scale bar and scale text
  im_c = closing(im, disk(4))

  # Ensure no data is lost in the closing step by enlarging im_c and performing boolean logic with original image to isolate tepals
  im_c_overlap = erode(im_c, disk(7))
  im_bool = img_as_bool(im)
  im_teps = np.invert(np.logical_and(np.invert(im_bool), np.invert(im_c_overlap)))

  # Label tepals
  label_teps, _ = scipylabel(np.invert(im_teps))

  # Isolate tepals
  tepals = []

  for tepal in regionprops(label_teps):
    crop = np.pad(tepal.image, np.round(0.05*len(tepal.image)).astype(int))
    tepals.append(crop)

  # Save tepals
  tepal1 = img_as_ubyte(tepals[0])
  tepal2 = img_as_ubyte(tepals[1])
  tepal3 = img_as_ubyte(tepals[2])

  return tepal1, tepal2, tepal3

#Acquire all tepal files into one array.

In [ ]:
# Directory in Lily Project SharedDrive that contains "Big White Lily" slices
dir_bigwhite = '/content/drive/Shareddrives/Keller Lab Lily Project/Slices "big white 15Oct2013"/'

# Sort images by number
files = []

substring = 'mask only'           # Substring to extract only filenames you want

for filename in os.listdir(dir_bigwhite):
  
  if substring in filename:       # Check if the file is a mask
    files.append(filename)

files.sort()

# To check if you're extracting the correct files
print(files)

In [ ]:
# Save original photos into tepal_photos array
tepal_names = []

substring2 = 'IMG_'

for name in os.listdir(dir_bigwhite):
  if substring2 in name:
    tepal_names.append(name)

tepal_names.sort()

tepal_photos = []

for k, photo_name in enumerate(tepal_names):
  current_photo = plt.imread(dir_bigwhite + photo_name)
  tepal_photos.append(current_photo)

In [ ]:
# Save images in list to an array
slices = []

for i, slice_name in enumerate(files):
  
  current_slice = plt.imread(dir_bigwhite + slice_name)   # Read file into current_slice

  if current_slice.ndim > 2:                              # Check dimensionality, and save only 2 dim array
    current_slice = current_slice[:,:,0]

  slices.append(current_slice)                            # Append to slices array

In [ ]:
# Initialize arrays for tepal slices
tepals1 = []
tepals2 = []
tepals3 = []

for nth, image in enumerate(slices):
  tepal1, tepal2, tepal3 = separate_tepals(slices[nth])   # Separate tepals

  print('Slice ' + str(nth) + ': Completed')

  tepals1.append(tepal1)    # Append to tepal slice arrays
  tepals2.append(tepal2)
  tepals3.append(tepal3)

Function Used within ```interact```





In [ ]:
def check_tepalStacks(index, mask_OR_photo=0):

  fig, ax = plt.subplots(ncols=4, figsize=(24,6))

  if mask_OR_photo == 0:
    ax[0].imshow(slices[index], cmap='gray')
  else:
    ax[0].imshow(tepal_photos[index + 1], cmap='hsv')

  ax[1].imshow(tepals2[index], cmap='gray')
  ax[2].imshow(tepals1[index], cmap='gray')
  ax[3].imshow(tepals3[index], cmap='gray')

  ax[0].set_title(str(files[index]))
  ax[1].set_title('tepals2: Slice ' + str(index))
  ax[2].set_title('tepals1: Slice ' + str(index))
  ax[3].set_title('tepals3: Slice ' + str(index))

  for axi in ax:
    axi.axis('off')

In [ ]:
interact(check_tepalStacks, index=widgets.IntSlider(value=0, min=0, max=len(slices)-1, step=1), mask_OR_photo=widgets.IntSlider(value=0, min=0, max=1, step=1))

interactive(children=(IntSlider(value=0, description='index', max=43), IntSlider(value=0, description='mask_OR…

<function __main__.check_tepalStacks>

# (EXAMPLE) To me, it looks like we need to swap the images for the following tepals:
Sets 1 and 2: 12, 13, 18, 23, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 39, 40, 42, 43

Sets 2 and 3: 16.

Sets 1 and 3: None.

Here's a quick function for that. Just generate a list of the indices you want to switch, and pass them into the function below along with your image lists.

In [ ]:
def switch_tepals(switch_indices, tepal_list1, tepal_list2):

  # Switch tepals at given indices between tepal_list1 and tepal_list2
  for index in switch_indices:

    # Save tepal images that will be switched to two intermediate variables before overwriting
    temp_tepal1 = tepal_list1[index]
    temp_tepal2 = tepal_list2[index]

    # Switch incorrect tepal for correct tepal in both arrays
    tepal_list1[index] = temp_tepal2
    tepal_list2[index] = temp_tepal1
  
  return tepal_list1, tepal_list2

In [ ]:
# Switch indices for tepals1 and tepals2
switch_1and2 = [12,13,18,23,27,28,29,30,31,32,34,35,36,38,39,40,42,43]

tepals1, tepals2 = switch_tepals(switch_1and2, tepals1, tepals2)

In [ ]:
# Switch indices for tepals2 and tepals3
switch_2and3 = [16]

tepals2, tepals3 = switch_tepals(switch_2and3, tepals2, tepals3)